In [1]:
import json
import copy

In [2]:
# pretty prints
from pprint import pprint

In [3]:
# creating directory for json configs
import os

if not os.path.isdir("gobot"):
    os.mkdir("gobot")

In [4]:
import deeppavlov

**NOTE**: "go_bot" model trains faster on a CPU, so let's ignore existing GPUs:

In [5]:
!export CUDA_VISIBLE_DEVICES=""
!echo "cuda visible devices = '"$CUDA_VISIBLE_DEVICES"'"

cuda visible devices = ''


# Hybrid goal-oriented bot

Dialog bots are categorized into two types:

1. **goal-oriented models **

    (those who have to achieve some kind of a goal in the end of conversation: 
     - restaurant and flight booking,
     - customer support service,
     - etc.);
 
2. **chit-chat models **

   (those who chat just for fun, the longer bot speaks with you the better, example:
    - "replica" mobile application).

We will only dive into goal-oriented task specification.

![go bot architecture 00](img/bot_architecture00.png)

A classical dialog system consists of:

1. **Natural Language Understanding component (NLU)**

 that is intended to "understand" human and represent it's "understanding" in a machine readable format. 

 It takes an utterance text as input and converts to a dialog "frame".

 "Frame" may consist of:
   - domain value (domain is some kind of "a type of dialogs");
   - intent value (intent of current human utterance: "welcome_message", "asking_weather", etc.);
   - entity slots (entities are mentioned by human "location", "time", etc.).

2. **Dialogue Manager component (DM)**

 that is intended to decide what to respond. 
    
 It takes a filled by NLU frame and outputs action (it isn't a final text, it is a label). 
    
 For example, there may be actions: "say_welcome", "say_goodbye", "ask_location", "give_weather", etc.

3. **Natural Language Generation component(NLG)**

 that is intended to convert action to an actual text response representation.
 
 For example, "say_goodbye" -> "You are welcome!".

## NLU 

![go bot architecture 01](img/bot_architecture01.png)

Let's consider a dialog system with NLU component that consists of a single Named Entity Recognition 
    component (or NER).

One of the previous tutorials introduced deeppavlov NER model and showed how to use it.

## DM & NLG

The tutorial is focused on how to implement
 - Dialogue Manager and
 - Natural Language Generator.

### Dataset

We will train the chatbot on a [Dialog State Tracking Chellenge 2](http://camdial.org/~mh521/dstc/) data.

Let's download it first.

In [7]:
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader

data = DSTC2DatasetReader().read(data_path="tmp/my_download_of_dstc2")

2018-08-22 11:44:34.195 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 96: [downloading data from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to tmp/my_download_of_dstc2]
2018-08-22 11:44:34.198 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-08-22 11:44:34.360 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz HTTP/1.1" 200 506300
2018-08-22 11:44:34.365 INFO in 'deeppavlov.core.data.utils'['utils'] at line 62: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to tmp/my_download_of_dstc2/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 51.0MB/s]
2018-08-22 11:44:34.378 INFO in 'deeppavlov.core.data.utils'['utils'] at line 200: Extracting tmp/my_download_of_dstc2/dstc2_v2.tar.gz archive into tmp/my_download_of_dstc2
2018-08-22 11:44:34.401 INFO in 'deeppavlov.dataset_reader

`DSTC2DatasetReader` downloaded the needed data and saved to disk.

`DialogDatasetIterator` took the data as input and transformed it to batches.

In [8]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

batches_generator = DialogDatasetIterator(data, seed=1443, shuffle=True)\
                                         .gen_batches(batch_size=4, data_type='train')

-------------
-------------
   Let's take a closer look at a batch content:

In [9]:
batch = batches_generator.__next__()

Each batch is a tuple of two elements:
  - list of x's and
  - list of y's

In [10]:
x_batch, y_batch = batch

`x_batch` (and `y_batch`) consists of 4 samples. This is because `batch_size` was 4.

In [11]:
len(x_batch)

4

One sample is a dialog. This is how one turn from a dialog looks like:

In [12]:
dialog_id = 0
dialog = [(x, y) for x, y in zip(x_batch[dialog_id], y_batch[dialog_id])]

turn_id = 0
print("----{}th turn----".format(turn_id)) 
pprint(dialog[turn_id], indent=8)

----0th turn----
(       {'intents': [], 'prev_resp_act': None, 'text': ''},
        {       'act': 'welcomemsg',
                'text': 'Hello, welcome to the Cambridge restaurant system. '
                        'You can ask for restaurants by area, price range or '
                        'food type. How may I help you?'})


This is how the whole dialog looks (printing `'text'` parts of `x_batch` and `y_batch`):

In [13]:
for turn in dialog:
    x, y = turn
    print('::', x['text']) 
    print('>>', y['text'], '\n')

:: 
>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you? 

:: traditional
>> api_call area="dontcare" food="traditional" pricerange="dontcare" 

:: traditional
>> I am sorry but there is no traditional restaurant that matches your request. 

:: italian food
>> What part of town do you have in mind? 

:: south
>> api_call area="south" food="italian" pricerange="dontcare" 

:: south
>> Pizza hut cherry hinton is a nice place in the south of town serving tasty italian food. 

:: phone number
>> The phone number of pizza hut cherry hinton is 01223 323737. 

:: post code
>> The post code of pizza hut cherry hinton is C.B 1, 7 D.Y. 

:: thank you good bye
>> You are welcome! 



### HowTo: DeepPavlov configs

DeepPavlov uses json configs as a tool to configure data pipelines.

Config has the following sections:
     
 - **dataset_reader**
   - configuration of dataset reader component (is responsible for data download and saving to disk);
  
 - **dataset_iterator**
   - configuration of dataset iterator component (is responsible for making batches (sequences) of data that will be further fed to pipe components);
  
 - **metadata**
   - extra info (urls for data download and telegram configuration);

 - **train**
   - training process configuration (size of batches, number of training epochs, etc.);
   
 - **chainer**
   - specifies data flow (which components are run and in what order);

Let's construct a simple config that builds a dictionary of input sample tokens.

In [14]:
vocab_config = {}

- **dataset_reader** -- configuration of dataset reader component (that is responsible for data download and saving to disk)

In [15]:
dstc2_reader_comp_config = {
    'name': 'dstc2_reader',
    'data_path': 'dstc2'
}

In [16]:
vocab_config['dataset_reader'] = dstc2_reader_comp_config

- **dataset_iterator** -- configuration of dataset iterator component (that is responsible for making batches (sequences) of data that will be further fed to pipe components)

In [17]:
dialog_iterator_comp_config = {
    'name': 'dialog_iterator'
}

In [18]:
vocab_config['dataset_iterator'] = dialog_iterator_comp_config

- **metadata** -- some extra info
     - **metadata.download** -- a list of data which should be downloaded in order for config to work

In [19]:
dstc2_download_config = {
    'url': 'http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz',
    'subdir': 'dstc2'
}

In [20]:
vocab_config['metadata'] = {}
vocab_config['metadata']['download'] = [
    dstc2_download_config
]

- **train** -- training process configuration
     
We don't need to train anything now, just build (fit on whole dataset once) a dictionary, so "train" section is empty.

In [21]:
vocab_config['train'] = {}

 - **chainer** specifies data flow:
 
     - **chainer.in** -- is a list of input sample names (one data sample might consist of several variables);
     - **chainer.in_y** -- is a list of input label names (each sample might have labels of different kind);
     - **chainer.out** -- is a list of output prediction names (usually has the same length as "chainer.in_y");
     
X is only an utterance here.

Y is empty (we don't need to train the dictionary like neural networks)

There is no prediction for the config, nothing to predict.

In [22]:
vocab_config['chainer'] = {}
vocab_config['chainer']['in'] = ['utterance']
vocab_config['chainer']['in_y'] = []
vocab_config['chainer']['out'] = []

- **chainer**
     - **chainer.pipe** -- is a list of consequently run components. This is the place where you specify in which order and what kind of data will be fed to components. 
     
Our pipe consists of one component -- "default_vocab".

##### HowTo: Component config

Component configs are always just a part of a global model config (described above).

Config for any component contains the following **_required_** parameters:
 - **name** -- registered name of a component (it is a link to python component implementation)
 - **save_path** -- path to save the component (sometimes is not needed, for example, for tokenizers)
 - **load_path** -- path to load the component (sometimes is not needed, for examples, for tokenizers)

and the following **_optional_** parameters:
 - **id** -- reference name for the component
 - **ref** -- "id" of a component that was previously initialized. It can be used instead of "name".
 - **fit\_on** -- a list of data fields to fit on (it calls \_\_fit\_\_ method of the component)
 - **in** -- a list of data fields that are inputs during inference (prediction)
 - **out** -- a list of data fields that are outputs during inference (prediction)
 
 
"default_vocab" component also has it's on unique parameters:
 - level -- on which level to operate ('token' level and 'char' (character) level are available)
 - tokenizer -- if input is a string, then it will be tokenized by the tokenizer, _optional parameter_

In [23]:
vocab_comp_config = {
    'name': 'default_vocab',
    'save_path': 'vocabs/token.dict',
    'load_path': 'vocabs/token.dict',
    'fit_on': ['utterance'],
    'level': 'token',
    'tokenizer': {'name': 'split_tokenizer'},
    'main': True
}

In [24]:
vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]

In [25]:
json.dump(vocab_config, open("gobot/vocab_config.json", 'wt'))

To download "dstc2_v2" dataset use `deeppavlov.deep_download` script (you have to do it only once):

In [26]:
from deeppavlov.download import deep_download # it is called "deep" in honor of "Deep Pavlov"

deep_download('gobot/vocab_config.json')

2018-08-22 11:45:57.104 INFO in 'deeppavlov.download'['download'] at line 112: Downloading...
2018-08-22 11:45:57.105 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-08-22 11:45:57.109 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /datasets/dstc2_v2.tar.gz HTTP/1.1" 200 506300
2018-08-22 11:45:57.109 INFO in 'deeppavlov.core.data.utils'['utils'] at line 62: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/ipavlov/Pilot/download/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 36.8MB/s]
2018-08-22 11:45:57.124 INFO in 'deeppavlov.core.data.utils'['utils'] at line 200: Extracting /home/vimary/ipavlov/Pilot/download/dstc2_v2.tar.gz archive into /home/vimary/ipavlov/Pilot/download/dstc2
2018-08-22 11:45:57.148 INFO in 'deeppavlov.download'['download'] at line 114: 
Download successful!


All data and models are saved to root of deeppavlov module + `../download` (`DEEPPAVLOV_ROOT/../download/`).

In [28]:
dstc2_path = deeppavlov.__path__[0] + '/../download/dstc2'

Data was downloaded to `dstc2_path`:

In [29]:
# The command will only work for linux, do not panic otherwise -- it isn't something crucially important.
# You can further just comment bash commands.
!echo "> ls $dstc2_path"
!ls $dstc2_path

> ls /home/vimary/ipavlov/Pilot/deeppavlov/../download/dstc2
dstc2-templates.txt  dstc2-tst.jsonlist  resto.sqlite
dstc2-trn.jsonlist   dstc2-val.jsonlist


Let's build our vocabulary.

In [30]:
from deeppavlov.core.commands.train import train_evaluate_model_from_config

train_evaluate_model_from_config("gobot/vocab_config.json")

2018-08-22 11:46:20.126 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-trn.jsonlist]
2018-08-22 11:46:20.370 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-val.jsonlist]
2018-08-22 11:46:20.523 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-tst.jsonlist]
2018-08-22 11:46:20.703 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:46:20.722 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:46:20.723 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vi

{"valid": {"eval_examples_count": 575, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 576, "metrics": {"accuracy": 0.0}, "time_spent": "0:00:01"}}


Vocabulary was built on data and saved to disk.

`save_path = 'vocabs/token.dict'` and component files are saved to `DEEPPAVLOV_ROOT/../download/vocabs/token.dict`.

In [31]:
vocabs_path = deeppavlov.__path__[0] + '/../download/vocabs'

In [32]:
!echo "> ls $vocabs_path"
!ls $vocabs_path

> ls /home/vimary/ipavlov/Pilot/deeppavlov/../download/vocabs
ag_news_classes.dict		src_tokens2.dict
char.dict			src_tokens4.dict
classes.dict			src_tokens6.dict
dstc2_classes.dict		src_tokens7.dict
einav_classes.dict		src_tokens.dict
insults_kaggle_classes.dict	tag.dict
kvret_src_tokens.dict		tgt_tokens2.dict
kvret_tgt_tokens.dict		tgt_tokens4.dict
sentiment_twitter_classes.dict	tgt_tokens6.dict
sentirueval_all_classes.dict	tgt_tokens7.dict
sentirueval_bank_classes.dict	tgt_tokens.dict
sentirueval_car_classes.dict	token.dict
sentirueval_rest_classes.dict	toxic_challenge_classes.dict
sentirueval_ttk_classes.dict	word.dict
snips_classes.dict


This is the content of the saved "token.dict":

In [33]:
!echo "> head $vocabs_path/token.dict"
!head $vocabs_path/token.dict

> head /home/vimary/ipavlov/Pilot/deeppavlov/../download/vocabs/token.dict
cheap	391
restaurant	1179
any	259
south	306
address	781
phone	819
number	825
thank	998
you	1055
good	891


##### Using trained component

We can use built vocabulary by initializing it with `build_model`.

We need to add `in` and `out` to component configuration ( to know what are inputs and outputs during prediction ) :
 - **in** -- a list of data fields that are inputs during inference (prediction)
 - **out** -- a list of data fields that are outputs during inference (prediction)

In [34]:
vocab_comp_config['in'] = ['utterance']
vocab_comp_config['out'] = ['utterance_token_indices']

vocab_config['chainer']['pipe'] = [
    vocab_comp_config
]
vocab_config['chainer']['out'] = ['utterance_token_indices']

In [35]:
from deeppavlov import build_model

model = build_model(vocab_config)

2018-08-22 11:46:27.682 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]


Model expects a list of samples (batch) as input.

In [36]:
model(['hi'])

[141]

### Model `gobot_dstc2_simple`

Now let's train a simple goal-oriented bot:

In [37]:
from deeppavlov import build_model, deep_download, train_evaluate_model_from_config

"dataset_reader", "dataset_iterator" and "metadata" will be the same as for vocabulary only.

In [38]:
simple_config = {}

simple_config['dataset_reader'] = dstc2_reader_comp_config
simple_config['dataset_iterator'] = dialog_iterator_comp_config
simple_config['metadata'] = {}
simple_config['metadata']['download'] = [
    dstc2_download_config
]

X here is a dict `'x'` containing context 'text', 'intents', db_result', 'prev_resp_act'

Y here is a dict `'y'` containing response 'act' and 'text'

Prediction `'y_predicted'` here will be only 'text'

In [39]:
simple_config['chainer'] = {}
simple_config['chainer']['in'] = ['x']
simple_config['chainer']['in_y'] =['y']
simple_config['chainer']['out'] = ['y_predicted']

The bot consists (`pipe` section) of two components:
- **`default_vocab`** (or DefaultVocabulary) component that 

    - remembers all tokens from user utterances. 
    - `DefaultVocabulary.__call__` method inputs batch of tokens and outputs their indeces.

Vocabulary component will be the same as before, but let's add reference to component using `id` 
- **id** -- reference name for the component

In [40]:
vocab_comp_config = {
    'name': 'default_vocab',
    'id': 'token_vocab',
    'load_path': 'vocabs/token.dict',
    'save_path': 'vocabs/token.dict',
    'fit_on': ['x'],
    'level': 'token',
    'tokenizer': {'name': 'split_tokenizer'}
}

Adding vocabulary to chainer:

In [41]:
simple_config['chainer']['pipe'] = []
simple_config['chainer']['pipe'].append(vocab_comp_config)

- **`go_bot`** (or GoalOrientedBot) component that
    - calls `slot_filler` that for user utterance outputs mentioned slots 
        (for example, "i want cheap food" -> {'pricerange': 'cheap'})
    - updates dialog state with `tracker` (DialogStateTracker)
        
          (for example, if old state was {'location': 'north'}, 
          and current slots are {'pricerange': 'cheap'}, 
          then new dialog state will be {'location': 'north', 'pricerange': 'cheap'})
    - converts user utterance in string format (`x`) to tokens with `tokenizer`

          (for example, "hi, i want some cheap food" -> ['hi', ',', 'i', 'want', 'some', 'cheap', 'food'])
    - then embeds the tokens with bag-of-words using `bow_embedder`(if not None) and `word_vocab`

          (for example, "cheap" -> [1, 0, 0, 0, .., 0])
    - embeds the utterance with continuous `embedder` (if not None) as a mean of embeddings of utterance tokens
        
          (for example, "i" -> [0.1231, 0.23423, .., 0.03489])
    - concatenates embeddings and passes it as an input to a recurrent neural network (RNN)
    - trains RNN (with LongShortTermMemory (LSTM) as a core graph) that outputs an action label
    - loads templates (mapping from labels to string) using `template_path` and `template_type` and converts action label to string
        
          (for example, "bye_msg" -> "You are welcome!")
    - fills result string with slot values from dialog state
        
          (for example, if
           dialog state is equal to {'pricerange': 'cheap'}
           and output string is "There are no restaurants in a #pricerange pricerange"
           then the result response will be "There are no restaurants in a cheap pricerange")

In [42]:
bot_comp_config = {
    'name': 'go_bot',
    'in': ['x'],
    'in_y': ['y'],
    'out': ['y_predicted'],
    'word_vocab': None,
    'bow_embedder': {"name": "bow"},
    'embedder': None,
    'slot_filler': None,
    'template_path': 'dstc2/dstc2-templates.txt',
    'template_type': 'DualTemplate',
    'database': None,
    'api_call_action': 'api_call',
    'network_parameters': {
      'load_path': 'gobot_dstc2_simple/model',
      'save_path': 'gobot_dstc2_simple/model',
      'dense_size': 64,
      'hidden_size': 128,
      'learning_rate': 0.002,
      'attention_mechanism': None
    },
    'tokenizer': {'name': 'stream_spacy_tokenizer',
                  'lowercase': False},
    'tracker': {'name': 'featurized_tracker',
                'slot_names': ['pricerange', 'this', 'area', 'food', 'name']},
    'main': True,
    'debug': False
}

This is how we use vocabulary by reference:

In [43]:
bot_comp_config['word_vocab'] = '#token_vocab'

Announcing slot filler component.
We assume that slot filler is already trained, and use it by referencing it's config.

In [44]:
slot_filler_comp_config = {
    'config_path': deeppavlov.__path__[0] + '/../deeppavlov/configs/ner/slotfill_dstc2.json'
}

Adding slot filler to bot component:

In [45]:
bot_comp_config['slot_filler'] = slot_filler_comp_config

Adding `bot_comp_config` to `pipe`:

In [46]:
simple_config['chainer']['pipe'].append(bot_comp_config)

Neural network (in the bot) is trained in epochs, and needs data in the form of batches.

That is why we are now filling "train" section with training parameters.

- **train** -- training process configuration
     - **train.batch_size** is a number of samples in a batch (feeded to the network during one training step)
     - **train.epochs** is a number of iterations over dataset during training
     - **train.log_every_n_batches** and **train.log_every_n_epochs** control frequency of logging messages
     - **train.metrics** is a list of metrics used to validate our performance
     - **train.val_every_n_batches** and **train.val_every_n_epochs** describes how often we calculate metrics on `valid` data split
     - **train.validation_patience** is a number of epochs without metric improvement on `valid` data that we are able to endure =)

In [47]:
simple_bot_train_config = {
    'batch_size': 4,
    'epochs': 2,
    'log_every_n_batches': -1,
    'log_every_n_epochs': 1,
    'metrics': ['per_item_dialog_accuracy'],
    'val_every_n_epochs': 1,
    'validation_patience': 20
}

In [48]:
simple_config['train'] = simple_bot_train_config

In [49]:
json.dump(simple_config, open("gobot/simple_config.json", 'wt'))

`train.epochs` is set to '2' for now, if you intend to train a smarter model, you should increase it (a range from 10 to 200 epochs is recommended).

In [50]:
deep_download(slot_filler_comp_config['config_path'])

2018-08-22 11:46:56.795 INFO in 'deeppavlov.download'['download'] at line 112: Downloading...
2018-08-22 11:46:56.798 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 205: Starting new HTTP connection (1): files.deeppavlov.ai:80
2018-08-22 11:46:56.834 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 393: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/slotfill_dstc2.tar.gz HTTP/1.1" 200 640674
2018-08-22 11:46:56.835 INFO in 'deeppavlov.core.data.utils'['utils'] at line 62: Downloading from http://files.deeppavlov.ai/deeppavlov_data/slotfill_dstc2.tar.gz to /home/vimary/ipavlov/Pilot/slotfill_dstc2.tar.gz
100%|██████████| 641k/641k [00:00<00:00, 46.8MB/s]
2018-08-22 11:46:56.850 INFO in 'deeppavlov.core.data.utils'['utils'] at line 200: Extracting /home/vimary/ipavlov/Pilot/slotfill_dstc2.tar.gz archive into /home/vimary/ipavlov/Pilot/download
2018-08-22 11:46:56.859 INFO in 'deeppavlov.download'['download'] at line 114: 
Download successful!


In [52]:
train_evaluate_model_from_config("gobot/simple_config.json")

2018-08-22 11:50:36.720 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-trn.jsonlist]
2018-08-22 11:50:37.45 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-val.jsonlist]
2018-08-22 11:50:37.131 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-tst.jsonlist]
2018-08-22 11:50:37.352 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:50:37.373 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:50:40.746 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 93: [loading vocabulary f

{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.3629}, "time_spent": "0:00:51"}}


2018-08-22 11:51:53.443 INFO in 'deeppavlov.core.commands.train'['train'] at line 432: New best per_item_dialog_accuracy of 0.4009
2018-08-22 11:51:53.443 INFO in 'deeppavlov.core.commands.train'['train'] at line 434: Saving model
2018-08-22 11:51:53.443 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 50: [saving model to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_simple/model]
2018-08-22 11:51:53.507 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 353: [saving parameters to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_simple/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4009}, "time_spent": "0:01:11", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}


2018-08-22 11:52:44.858 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 371: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.5086}, "time_spent": "0:02:03"}}


2018-08-22 11:53:04.382 INFO in 'deeppavlov.core.commands.train'['train'] at line 432: New best per_item_dialog_accuracy of 0.4564
2018-08-22 11:53:04.383 INFO in 'deeppavlov.core.commands.train'['train'] at line 434: Saving model
2018-08-22 11:53:04.383 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 50: [saving model to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_simple/model]
2018-08-22 11:53:04.435 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 353: [saving parameters to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_simple/model.json]
2018-08-22 11:53:04.438 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:53:04.439 WARNING in 'deeppavlov.core.commands.infer'['infer'] at line 48: No "save_path" parameter for the go_bot component, so "load_path" will not be renewed
2018-08-22 11:53:04.441 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4564}, "time_spent": "0:02:22", "epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "impatience": 0, "patience_limit": 20}}


2018-08-22 11:53:05.167 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 41: [loading model from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/model]
2018-08-22 11:53:05.180 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/model
2018-08-22 11:53:05.434 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-08-22 11:53:05.435 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-templates.txt]
2018-08-22 11:53:05.436 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-08-22 11:53:05.436 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 137: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-08-22 11:53:05.907 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 119: [initializing `GoalOrientedBot

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4564}, "time_spent": "0:00:21"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4454}, "time_spent": "0:00:20"}}


Let's comminicate with the resulting bot. "exit" message initiates end of dialogue.

In [53]:
model = build_model(simple_config)

2018-08-22 11:53:46.160 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:53:46.162 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 93: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/word.dict]
2018-08-22 11:53:46.164 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 93: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/tag.dict]
2018-08-22 11:53:46.725 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 41: [loading model from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/model]
2018-08-22 11:53:46.739 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/model
2018-08-22 11:53:47.208 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-08

In [54]:
model(['hi, i want some cheap italian food in the north of town'])

2018-08-22 11:53:47.809 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 386: No database specified.
2018-08-22 11:53:47.809 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 387: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [55]:
model(['thanks, bye'])

2018-08-22 11:53:47.824 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 386: No database specified.
2018-08-22 11:53:47.824 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 387: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


['Sorry there is no italian restaurant in the north of town.']

In [56]:
model.reset() # resetting dialog context to start a new one

In [58]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

2018-06-28 14:46:37.496 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-06-28 14:46:37.497 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'italian', 'area': 'north'}, got 0 results.


>> Sorry there is no italian restaurant in the north of town.
:: i want french food


2018-06-28 14:46:51.212 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-06-28 14:46:51.212 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'french', 'area': 'north'}, got 0 results.


>> Sorry there is no french restaurant in the north of town.
:: ok, bye


2018-06-28 14:46:54.950 WARNING in 'deeppavlov.models.go_bot.bot'['bot'] at line 343: No database specified.
2018-06-28 14:46:54.950 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 344: Made api_call with {'pricerange': 'cheap', 'food': 'french', 'area': 'north'}, got 0 results.


>> Sorry there is no french restaurant in the north of town.
:: exit


The model couldn't fill some slots. For example, #address, #phone, #postcode of a restaurant couldn't be inferred from user utterance. 

A list of available restaurants is required.

### Model `gobot_dstc2_db`

Now let's now add a database with restaurants and train a new model:

Initializing new config:

In [57]:
db_config = copy.deepcopy(simple_config)

db_config['chainer']['pipe'] = []

Creating database component config:

In [58]:
db_comp_config = {
    'name': 'sqlite_database',
    'id': 'restaurant_database', 
    'save_path': 'dstc2/resto.sqlite',
    'primary_keys': ['name'],
    'table_name': 'mytable'
}

Adding vocab and database components to pipe:

In [59]:
db_config['chainer']['pipe'].append(vocab_comp_config)
db_config['chainer']['pipe'].append(db_comp_config)

Initializing bot component config:

In [60]:
bot_with_db_comp_config = copy.deepcopy(bot_comp_config)

**WARNING:** Do no forget to change `load_path` and `save_path` in neural network configuration when 
             training a new modification. Otherwise previous model's files will be overwritten.

In [61]:
bot_with_db_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_db/model'
bot_with_db_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_db/model'

Adding database to bot component config:

In [62]:
bot_with_db_comp_config['database'] = '#restaurant_database'

Addind bot component to pipe:

In [63]:
db_config['chainer']['pipe'].append(bot_with_db_comp_config)

In [64]:
json.dump(db_config, open("gobot/db_config.json", 'wt'))

The new model now updates dialog state not only with entity values mentioned by user ("i want cheap food" -> {'pricerange': 'cheap'}), but also with restaurant info taken from sql database of restaurants.

Model has a special action `api_call_action`, which initiates a request to sql database with current dialog state and thus receives info of a single matching restaurant.

So now such slots as #address, #phone and #postcode can be filled in bot responses.

In [65]:
train_evaluate_model_from_config("gobot/db_config.json")

2018-08-22 11:58:37.511 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-trn.jsonlist]
2018-08-22 11:58:37.632 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-val.jsonlist]
2018-08-22 11:58:37.936 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 113: [loading dialogs from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-tst.jsonlist]
2018-08-22 11:58:38.31 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:58:38.49 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 163: [saving vocabulary to /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 11:58:38.52 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is se

{"train": {"epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "metrics": {"per_item_dialog_accuracy": 0.4076}, "time_spent": "0:00:51"}}


2018-08-22 11:59:48.610 INFO in 'deeppavlov.core.commands.train'['train'] at line 432: New best per_item_dialog_accuracy of 0.4197
2018-08-22 11:59:48.611 INFO in 'deeppavlov.core.commands.train'['train'] at line 434: Saving model
2018-08-22 11:59:48.611 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 50: [saving model to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_db/model]
2018-08-22 11:59:48.675 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 353: [saving parameters to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_db/model.json]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4197}, "time_spent": "0:01:10", "epochs_done": 1, "batches_seen": 242, "train_examples_seen": 967, "impatience": 0, "patience_limit": 20}}


2018-08-22 12:00:37.371 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 371: Updating global step, learning rate = 0.002000.


{"train": {"epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "metrics": {"per_item_dialog_accuracy": 0.5314}, "time_spent": "0:01:58"}}


2018-08-22 12:00:56.683 INFO in 'deeppavlov.core.commands.train'['train'] at line 432: New best per_item_dialog_accuracy of 0.4306
2018-08-22 12:00:56.683 INFO in 'deeppavlov.core.commands.train'['train'] at line 434: Saving model
2018-08-22 12:00:56.683 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 50: [saving model to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_db/model]
2018-08-22 12:00:56.739 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 353: [saving parameters to /home/vimary/ipavlov/Pilot/download/gobot_dstc2_db/model.json]
2018-08-22 12:00:56.743 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 12:00:56.744 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 12:00:56.744 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_data

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4306}, "time_spent": "0:02:18", "epochs_done": 2, "batches_seen": 484, "train_examples_seen": 1934, "impatience": 0, "patience_limit": 20}}


2018-08-22 12:00:57.564 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 41: [loading model from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/model]
2018-08-22 12:00:57.577 INFO in 'tensorflow'['tf_logging'] at line 115: Restoring parameters from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/model
2018-08-22 12:00:57.834 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 56: No load path is set for GoalOrientedBot!
2018-08-22 12:00:57.835 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 108: [loading templates from /home/vimary/ipavlov/Pilot/download/dstc2/dstc2-templates.txt]
2018-08-22 12:00:57.835 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 111: 46 templates loaded
2018-08-22 12:00:57.836 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 137: Calculated input size for `GoalOrientedBotNetwork` is 511
2018-08-22 12:00:58.291 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 119: [initializing `GoalOrientedBot

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4306}, "time_spent": "0:00:20"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4173}, "time_spent": "0:00:20"}}


In [66]:
model = build_model(db_config)

2018-08-22 12:01:37.242 INFO in 'deeppavlov.core.data.vocab'['vocab'] at line 175: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/vocabs/token.dict]
2018-08-22 12:01:37.243 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 53: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2018-08-22 12:01:37.244 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 63: Loading database from /home/vimary/ipavlov/Pilot/download/dstc2/resto.sqlite.
2018-08-22 12:01:37.245 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 93: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/word.dict]
2018-08-22 12:01:37.248 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 93: [loading vocabulary from /home/vimary/ipavlov/Pilot/download/slotfill_dstc2/tag.dict]
2018-08-22 12:01:37.796 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 41: [loading model from 

In [67]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

>> Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?
:: i want cheap restaurant in the north of town


2018-08-22 14:19:49.747 INFO in 'deeppavlov.models.go_bot.bot'['bot'] at line 387: Made api_call with {'pricerange': 'cheap', 'area': 'north'}, got 2 results.


>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: ok, give me their phone number
>> The phone number of da vinci pizzeria is 01223 351707.
:: post code too
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: give me their address
>> Sure, da vinci pizzeria is on 20 milton road chesterton.
:: bye then
>> Da vinci pizzeria is a nice place in the north of town and the prices are cheap.
:: exit


### Model `gobot_dstc2_emb`

Now let's train a goal-oriented bot with fasttext embeddings:

**NOTICE:** YOU NEED TO CONSTRUCT A NEW CONFIG YOURSELF

Initalizing new config:

In [68]:
emb_config = copy.deepcopy(db_config)

emb_config['chainer']['pipe'] = []

Adding vocab and database components to chainer pipe:

In [69]:
emb_config['chainer']['pipe'].append(vocab_comp_config)
emb_config['chainer']['pipe'].append(db_comp_config)

Initalizing embedder component:

In [70]:
embedder_comp_config = {
    'id': 'my_embedder',
    'name': 'fasttext',
    'load_path': 'embeddings/dstc2_fastText_model.bin',
    'save_path': 'embeddings/dstc2_fastText_model.bin',
    'dim': 100
}

In [71]:
# TODO: add embedder component to chainer pipe


Initializing bot component config:

In [72]:
bot_with_embedder_comp_config = copy.deepcopy(bot_with_db_comp_config)

bot_with_embedder_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_emb/model'
bot_with_embedder_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_emb/model'

In [73]:
# TODO: add #my_embedder to bot_with_embedder_comp_config


In [74]:
# TODO: add bot_with_embedder_comp_config to chainer pipe


These are download urls for new required data:

In [75]:
embedder_required_data = {
    'url': 'http://files.deeppavlov.ai/deeppavlov_data/embeddings/dstc2_fastText_model.bin',
    'subdir': 'embeddings'
}

In [76]:
# TODO: add embedder download info to emb_config['metadata']


In [77]:
json.dump(emb_config, open("gobot/emb_config.json", 'wt'))

As far as we are now using embeddings, we added a file named `dstc2_fastText_model.bin` to `metadata.download` section. 

Let's run data loading again.

In [ ]:
deep_download('gobot/emb_config.json')

In [ ]:
train_evaluate_model_from_config("gobot/emb_config.json")

In [ ]:
model = build_model(emb_config)

In [ ]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

## Appendix _(optional)_

### Additional materials

- [DataFest Presentation (RU)](https://docs.google.com/presentation/d/1PBPQp-wgQ6aRbm3MsuyGYB_TVg2c7Bf89lhdhbMtC2k)
- [CISS Video Lecture](https://youtu.be/uvH1zB7qahI)
- [Video Lecture "Hybrid dialog bot" (RU)](http://www.youtube.com/watch?v=JJCO7eWCy-M&t=331m19s)
- [Video Lecture "What's inside a dialog system?" (RU)](http://www.youtube.com/watch?v=JJCO7eWCy-M&t=259m55s)

### Model `gobot_dstc2_full`

Now let's train a very smart goal-oriented bot that uses an attention mechanism over input embeddings 

(see https://medium.com/syncedreview/a-brief-overview-of-attention-mechanism-13c578ba9129 for more details):

Initializing new config:

In [118]:
emb_config = json.load(open("gobot/emb_config.json", 'rt'))

full_config = copy.deepcopy(emb_config)
full_config['chainer']['pipe'] = [
    vocab_comp_config,
    db_comp_config,
    embedder_comp_config
]

Initializing bot component config:

In [119]:
bot_with_emb_comp_config = emb_config['chainer']['pipe'][-1]
bot_with_attn_comp_config = copy.deepcopy(bot_with_emb_comp_config)

bot_with_attn_comp_config['network_parameters']['load_path'] = 'gobot_dstc2_full/model'
bot_with_attn_comp_config['network_parameters']['save_path'] = 'gobot_dstc2_full/model'

Adding attention mechanism to bot:

In [120]:
attention_mechanism_config = {
    'action_as_key': True,
    'depth': 3,
    'hidden_size': 32,
    'max_num_tokens': 100,
    'projected_align': False,
    'type': 'cs_bahdanau'
}

In [121]:
bot_with_attn_comp_config['network_parameters']['attention_mechanism'] = attention_mechanism_config

Adding bot component to pipe:

In [122]:
full_config['chainer']['pipe'].append(bot_with_attn_comp_config)

In [123]:
json.dump(full_config, open("gobot/full_config.json", 'wt'))

In [ ]:
train_evaluate_model_from_config("gobot/full_config.json")

In [ ]:
model = build_model(full_config)

In [ ]:
# if the cell is running, please do not run other cells in parallel -- there is a possibility of a hangup

model.reset() # starting new dialog

utterance = ""
while utterance != 'exit':
    print(">> " + model([utterance])[0])
    utterance = input(':: ')

### Another way of training and infering a component

Let's build response token vocabulary, but do it without using deeppavlov scripts (without `train_evaluate_model_from_config` and `build_model`).

In [126]:
from deeppavlov.core.data.vocab import DefaultVocabulary
from deeppavlov.dataset_readers.dstc2_reader import DSTC2DatasetReader
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

Initializing a `DefaultVocabulary` class:

In [127]:
y_vocab = DefaultVocabulary(level='token', 
                            load_path='vocabs/y_token.dict', # path is relative to DEEPPAVLOV_ROOT/../download/ 
                            save_path='vocabs/y_token.dict',
                            tokenizer=lambda s_batch: [s.split() for s in s_batch])

Important methods of any trained component are:

   - **\_\_init\_\_(self, *args, *\*kwargs)**
     - intializes a class instance

   - **fit(self, data, *args)** or **train_on_batch(self, batch, *args)**
     - fits on full data or makes one training step on a batch of data

   - **\_\_call\_\_(self, batch, \*\*kwargs)**
     - makes prediction (or infers) for each sample in a batch

Getting batches of data:

In [ ]:
data = DSTC2DatasetReader().read(data_path="tmp/my_download_of_dstc2")
data_samples = DialogDatasetIterator(data, seed=1443, shuffle=True).get_instances(data_type='all')
x_list, y_list = data_samples

Building vocabulary using y batches (`y_list` contains bot responses):

In [129]:
y_vocab.fit(y_list)

Infering from (using) built vocabulary:

In [130]:
y_vocab(['is', 'the', 'of', 'restaurant', 'hi'])

[43, 3, 26, 5, 0]

To call a model `y_vocab(batch)` is the same as to call a \_\_call\_\_ method `y_vocab.__call__(batch)`!

In [198]:
y_vocab(['hi']) == y_vocab.__call__(['hi']) == [141]

False